<a href="https://colab.research.google.com/github/Thayyebsalim/music_analyisis/blob/master/Music_Sentimental_analysis_NLP_by_Tone_and_Lyrics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Intro

The idea of this code is to find the mood of the song in any way possible helping us to segregate songs better.

1.   First part is gonna be scraping the lyrics from any website or API 
2.   Finding the vibe of the song with sentimental analysis
3.   Fetch and mp3 version and check the mood with transfer learning hopefully or something similar



In [106]:
!apt-get update
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
!pip install selenium

Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran35/ InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:4 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:5 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Hit:6 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:10 http://ppa.launchpad.net/marutter/c2d4u3.5/ubuntu bionic InRelease
Get:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Fetched 252 kB in 2s (150 kB/s)
Reading package lists... Done
Reading package lists... Done
Building de

In [107]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
import re
import time
import string
from textblob import TextBlob
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
nltk.download("vader_lexicon")
nltk.download("brown")
nltk.download('stopwords')
nltk.download("wordnet")
nltk.download("punkt")

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Package brown is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

We need to scrape the song lyrics from websites or API to use here

I am using selenium here and the speed is not the best at the moment so I might switch to scrapy or use an API

In [108]:
chrome_options = Options()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')


def get_track(): # Gets track name from user and converts it so that we can use it in a link
  track_name = input("Enter a track name: ")
  track_name_link = track_name.rstrip().replace(" ","%20").lower()
  return track_name_link


def scrape_tracklist(track_link): # Scrapes the page for list of tracks with similar names or such 
  driver = webdriver.Chrome(chrome_options=chrome_options)
  driver.get("https://www.musixmatch.com/search/" + track_link + "/tracks")
  element = driver.find_element_by_class_name("box-content")
  link_list = [i.get_attribute('href') for i in element.find_elements_by_tag_name('a')]
  driver.close()
  lyric_links=[i for i in link_list if re.search("/lyrics/",i)] #could do it in one line but for the sake of clarity, I'm gonna do two
  lyric_links=[i for i in lyric_links if not re.search("/add",i)]
  return lyric_links


def scrape_lyrics(lyric_links_list,choice): # Scrapes the lyrics element
  driver = webdriver.Chrome(chrome_options=chrome_options)
  driver.get(lyric_links_list[choice])
  lyric_elem=driver.find_elements_by_class_name('mxm-lyrics__content')
  lyric_elem= [i.find_element_by_tag_name('span') for i in lyric_elem] 
  lyric_elem= [i.get_attribute('innerHTML') for i in lyric_elem]
  driver.close()
  return lyric_elem


def select_track(artist_track_list): # Lets the user select a track
  sel=0
  for track in artist_track_list:
    print(f"{track[0]+1} {track[1]} - {track[2]}")
  sel=int(input('Enter the number of your desired track: '))-1
  return sel


def get_lyric_element(tracklist,choice=0): # Cleans up the lyric element and returns a cleaned list(array) with just lyrics inside
  lyric_links_list = scrape_tracklist(track_link)
  artist_track_list = [[lyric_links_list.index(x), x.split("/")[4],x.split('/')[5]] for x in lyric_links_list]
  print(artist_track_list)
  if len(artist_track_list) > 1 and choice != "default":
    choice = select_track(artist_track_list)
  elif len(artist_track_list) == 0 :
    return "Sorry! no such song"
  else:
    choice=0
  lyric_element = scrape_lyrics(lyric_links_list,choice)
  return " ".join(lyric_element)



In [109]:
track_name= "have been the wind alec"
track_link=track_name.rstrip().replace(" ","%20").lower()
#track_link=get_track() #uncomment to activate function

In [110]:
lyric_element = get_lyric_element(track_link,"default")
lyric_element

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:14: DeprecationWarning: use options instead of chrome_options
  


[[0, 'Alec-Benjamin', 'Must-Have-Been-the-Wind-2'], [1, 'Alec-Benjamin', 'Must-Have-Been-the-Wind-1'], [2, 'Arcade-Player', 'Must-Have-Been-the-Wind-16-Bit-Alec-Benjamin-Emulation'], [1, 'Alec-Benjamin', 'Must-Have-Been-the-Wind-1'], [4, '8-Bit-Arcade', 'Must-Have-Been-the-Wind-8-Bit-Alec-Benjamin-Emulation']]


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:25: DeprecationWarning: use options instead of chrome_options


'I heard glass shatter on the wall in the apartment above mine\nAt first I thought that I was dreamin\'\nBut then I heard the voice of a girl And it sounded like she\'d been cryin\'\nNow I\'m too worried to be sleepin\'\n\nSo I took the elevator to the second floor\nWalked down the hall and then I knocked upon her door\nShe opened up and I asked about the things I\'ve been hearing\n\nShe said, "I think your ears are playing tricks on you"\nSweater zipped up to her chin\n"Thanks for caring, sir, that\'s nice of you\nBut I have to go back in\nWish I could tell you about the noise\nBut I didn\'t hear a thing"\nShe said, "It must have been the wind, must have been the wind\nMust have been the wind, it must have been the wind"\nShe said, "It must have been the wind, must have been the wind\nMust have been the wind, it must have been the wind"\n\nSo I was layin\' on the floor of my room\nCold concrete on my back\nNo, I just couldn\'t shake the feeling\nI didn\'t want to intrude because I kne

The first part is done, web scraper should work fine. I did not implement error cases for now becuase I want to focus on the next two parts of the project


In [111]:
def filter_raw_lyrics(raw_lyr): #The scraped element has blanks and \n's lets filter those 
  return [i for i in raw_lyr.split("\n") if i != '']

#joiner
def convert_arr_to_str(arr):
  return " ".join(arr)

In [112]:
lyrics_as_array= filter_raw_lyrics(lyric_element)
lyrics=convert_arr_to_str(lyrics_as_array) #returns a single string as lyrics
lyrics,lyrics_as_array

('I heard glass shatter on the wall in the apartment above mine At first I thought that I was dreamin\' But then I heard the voice of a girl And it sounded like she\'d been cryin\' Now I\'m too worried to be sleepin\' So I took the elevator to the second floor Walked down the hall and then I knocked upon her door She opened up and I asked about the things I\'ve been hearing She said, "I think your ears are playing tricks on you" Sweater zipped up to her chin "Thanks for caring, sir, that\'s nice of you But I have to go back in Wish I could tell you about the noise But I didn\'t hear a thing" She said, "It must have been the wind, must have been the wind Must have been the wind, it must have been the wind" She said, "It must have been the wind, must have been the wind Must have been the wind, it must have been the wind" So I was layin\' on the floor of my room Cold concrete on my back No, I just couldn\'t shake the feeling I didn\'t want to intrude because I knew that I didn\'t have all

Well lets have some fun with the lyrics

In [113]:
def swfilter(word): #function we use in filter()
  return word not in stopwords.words()

def remove_stopword(lyr_arr):
  return [" ".join(filter(swfilter,sentence.split())) for sentence in lyrics_as_array]

In [114]:
contractions = { 
"ain't": "am not / are not / is not / has not / have not",
"aren't": "are not / am not",
"can't": "cannot",
"can't've": "cannot have",
"'cause": "because",
"could've": "could have",
"couldn't": "could not",
"couldn't've": "could not have",
"didn't": "did not",
"doesn't": "does not",
"don't": "do not",
"hadn't": "had not",
"hadn't've": "had not have",
"hasn't": "has not",
"haven't": "have not",
"he'd": "he had / he would",
"he'd've": "he would have",
"he'll": "he shall / he will",
"he'll've": "he shall have / he will have",
"he's": "he has / he is",
"how'd": "how did",
"how'd'y": "how do you",
"how'll": "how will",
"how's": "how has / how is / how does",
"I'd": "I had / I would",
"I'd've": "I would have",
"I'll": "I shall / I will",
"I'll've": "I shall have / I will have",
"I'm": "I am",
"I've": "I have",
"isn't": "is not",
"it'd": "it had / it would",
"it'd've": "it would have",
"it'll": "it shall / it will",
"it'll've": "it shall have / it will have",
"it's": "it has / it is",
"let's": "let us",
"ma'am": "madam",
"mayn't": "may not",
"might've": "might have",
"mightn't": "might not",
"mightn't've": "might not have",
"must've": "must have",
"mustn't": "must not",
"mustn't've": "must not have",
"needn't": "need not",
"needn't've": "need not have",
"o'clock": "of the clock",
"oughtn't": "ought not",
"oughtn't've": "ought not have",
"shan't": "shall not",
"sha'n't": "shall not",
"shan't've": "shall not have",
"she'd": "she had / she would",
"she'd've": "she would have",
"she'll": "she shall / she will",
"she'll've": "she shall have / she will have",
"she's": "she has / she is",
"should've": "should have",
"shouldn't": "should not",
"shouldn't've": "should not have",
"so've": "so have",
"so's": "so as / so is",
"that'd": "that would / that had",
"that'd've": "that would have",
"that's": "that has / that is",
"there'd": "there had / there would",
"there'd've": "there would have",
"there's": "there has / there is",
"they'd": "they had / they would",
"they'd've": "they would have",
"they'll": "they shall / they will",
"they'll've": "they shall have / they will have",
"they're": "they are",
"they've": "they have",
"to've": "to have",
"wasn't": "was not",
"we'd": "we had / we would",
"we'd've": "we would have",
"we'll": "we will",
"we'll've": "we will have",
"we're": "we are",
"we've": "we have",
"weren't": "were not",
"what'll": "what shall / what will",
"what'll've": "what shall have / what will have",
"what're": "what are",
"what's": "what has / what is",
"what've": "what have",
"when's": "when has / when is",
"when've": "when have",
"where'd": "where did",
"where's": "where has / where is",
"where've": "where have",
"who'll": "who shall / who will",
"who'll've": "who shall have / who will have",
"who's": "who has / who is",
"who've": "who have",
"why's": "why has / why is",
"why've": "why have",
"will've": "will have",
"won't": "will not",
"won't've": "will not have",
"would've": "would have",
"wouldn't": "would not",
"wouldn't've": "would not have",
"y'all": "you all",
"y'all'd": "you all would",
"y'all'd've": "you all would have",
"y'all're": "you all are",
"y'all've": "you all have",
"you'd": "you had / you would",
"you'd've": "you would have",
"you'll": "you shall / you will",
"you'll've": "you shall have / you will have",
"you're": "you are",
"you've": "you have"
}

def map_contraction(word): #function we use in map()
  word=word.lower()
  if word in contractions:
    return contractions[word]
  else: 
    return word

def remove_contraction(lyr_arr):
  return [" ".join(map(map_contraction, sentence.split())) for sentence in lyr_arr]

In [115]:
wordnet_lemmatizer = WordNetLemmatizer()

def map_lemmatize(word):
  return wordnet_lemmatizer.lemmatize(word, pos="v")

def lemmatize_words(lyr_arr):
  return [" ".join(map(map_lemmatize, sentence.split())) for sentence in lyr_arr]

In [116]:
lyrics_as_array_without_sw = remove_stopword(lyrics_as_array)
lyrics_without_sw = convert_arr_to_str(lyrics_as_array_without_sw)
lyrics_as_array_without_sw,lyrics_without_sw

(['I heard glass shatter wall apartment',
  "At first I thought I dreamin'",
  "But I heard voice girl And sounded like she'd cryin'",
  "Now I'm worried sleepin'",
  'So I took elevator second floor',
  'Walked hall I knocked upon',
  "She opened I asked things I've hearing",
  'She said, "I think ears playing tricks you"',
  'Sweater zipped chin',
  '"Thanks caring, sir, that\'s nice',
  'But I go back',
  'Wish I could tell noise',
  'But I hear thing"',
  'She said, "It must wind, must wind',
  'Must wind, must wind"',
  'She said, "It must wind, must wind',
  'Must wind, must wind"',
  "So I layin' floor room",
  'Cold concrete back',
  'No, I shake feeling',
  'I intrude I knew I facts',
  "But I bear thought leavin'",
  'So I took elevator second floor',
  'Walked hall I knocked upon',
  "She opened I asked things I've hearing",
  'She said, "I think ears playing tricks you"',
  'Sweater zipped chin',
  '"Thanks caring, sir, that\'s nice',
  'But I go back',
  'Wish I could tell

In [117]:
lyrics_as_array_without_contraction = remove_contraction(lyrics_as_array)
lyrics_without_contraction = convert_arr_to_str(lyrics_as_array_without_contraction)
lyrics_as_array_without_contraction,lyrics_without_contraction

(['i heard glass shatter on the wall in the apartment above mine',
  "at first i thought that i was dreamin'",
  "but then i heard the voice of a girl and it sounded like she had / she would been cryin'",
  "now i'm too worried to be sleepin'",
  'so i took the elevator to the second floor',
  'walked down the hall and then i knocked upon her door',
  "she opened up and i asked about the things i've been hearing",
  'she said, "i think your ears are playing tricks on you"',
  'sweater zipped up to her chin',
  '"thanks for caring, sir, that has / that is nice of you',
  'but i have to go back in',
  'wish i could tell you about the noise',
  'but i did not hear a thing"',
  'she said, "it must have been the wind, must have been the wind',
  'must have been the wind, it must have been the wind"',
  'she said, "it must have been the wind, must have been the wind',
  'must have been the wind, it must have been the wind"',
  "so i was layin' on the floor of my room",
  'cold concrete on my

In [118]:
lyrics_as_array_with_lemmatized_words=lemmatize_words(lyrics_as_array)
lyrics_with_lemmatized_words=convert_arr_to_str(lyrics_as_array_with_lemmatized_words)
lyrics_as_array_with_lemmatized_words,lyrics_with_lemmatized_words

(['I hear glass shatter on the wall in the apartment above mine',
  "At first I think that I be dreamin'",
  "But then I hear the voice of a girl And it sound like she'd be cryin'",
  "Now I'm too worry to be sleepin'",
  'So I take the elevator to the second floor',
  'Walked down the hall and then I knock upon her door',
  "She open up and I ask about the things I've be hear",
  'She said, "I think your ears be play trick on you"',
  'Sweater zip up to her chin',
  '"Thanks for caring, sir, that\'s nice of you',
  'But I have to go back in',
  'Wish I could tell you about the noise',
  'But I didn\'t hear a thing"',
  'She said, "It must have be the wind, must have be the wind',
  'Must have be the wind, it must have be the wind"',
  'She said, "It must have be the wind, must have be the wind',
  'Must have be the wind, it must have be the wind"',
  "So I be layin' on the floor of my room",
  'Cold concrete on my back',
  "No, I just couldn't shake the feel",
  "I didn't want to intr


Next we have to tackle the sentimantal analysis part. We aim to try bag of words then use NLTK on every line might be slow but I like to give prominence to accuracy

In [119]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sia=SentimentIntensityAnalyzer()
def get_vader_score(lyr):
  # Polarity score returns dictionary
  sentiment = sia.polarity_scores(lyr)
  return sentiment['compound']
  """
  print(ss)
  print(ss['compound'])
  ss.pop("compound")
  print("ss=",ss)
  x=sorted(ss.values(), reverse=True)[0]
  for i in ss.keys():
    if ss[i]==x:
      if "i" == "neg":
        ss[i]=-(ss[i])
      print(ss[i])
  """  

In [120]:
get_vader_score(lyrics)

0.9599

In [121]:
def get_textblob_score(lyr):
  blob=TextBlob(lyr)
  return blob.polarity

In [122]:
get_textblob_score(lyrics)

0.14941520467836258

In [123]:
!pip install watson_developer_cloud
from watson_developer_cloud import NaturalLanguageUnderstandingV1
from watson_developer_cloud.natural_language_understanding_v1 import Features, EntitiesOptions, KeywordsOptions, SentimentOptions, CategoriesOptions
natural_language_understanding = NaturalLanguageUnderstandingV1(                                         
    version='2018-11-16',
    iam_apikey="hzkkbjivKNHfoi3cFlMBOPG6-YHLc-7CAQKwXn8Rt7JT", 
    url="https://api.eu-gb.natural-language-understanding.watson.cloud.ibm.com/instances/512aa79c-a8ba-43fa-b9d5-a4382099b06f" # paste the url here
    )
def get_watson_score(input_text): 
    response = natural_language_understanding.analyze (
    text = input_text,
    features = Features(sentiment=SentimentOptions())).get_result()
    res = response.get('sentiment').get('document').get('score')
    return res

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: DeprecationWarning: watson-developer-cloud moved to ibm-watson. To get updates, use the new package.
  import sys


In [124]:
get_watson_score(lyrics)

-0.778259

In [125]:
sentiment_score_fns=[get_textblob_score,get_vader_score]

In [126]:
# Returns all transformed arrays and further maps the arrays 

def get_lyric_arrays(lyr_arr):
  return [lyr_arr,
          remove_stopword(lyr_arr),
          remove_contraction(lyr_arr),
          lemmatize_words(lyr_arr),
          remove_contraction(remove_stopword(lyr_arr)),
          lemmatize_words(remove_stopword(lyr_arr)),
          remove_contraction(lemmatize_words(lyr_arr)),
          lemmatize_words(remove_contraction(remove_stopword(lyr_arr)))]

def get_lyrics(arr):
  return list(map(convert_arr_to_str,arr))

list_of_converted_lyr_arr=get_lyric_arrays(lyrics_as_array) #returns a tuple with 3 items lyrics, lyrics without stop wordsm 
list_of_converted_lyr=get_lyrics(list_of_converted_lyr_arr)

In [127]:
import pandas as pd 
import numpy as np

In [128]:
df=pd.DataFrame(index=["Lyrics","SW","CO","LM","SW-CO","SW-LM","CO-LM","SW-CO-LM"],columns=["Textblob","Textblob per line","Textblob nonzero","Vader","Vader per line","Vader nonzero","Watson","Watson per line","Watson nonzero"])
df

,Textblob,Textblob per line,Textblob nonzero,Vader,Vader per line,Vader nonzero,Watson,Watson per line,Watson nonzero
Lyrics,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
SW,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
CO,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
LM,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
SW-CO,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
SW-LM,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
CO-LM,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
SW-CO-LM,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [129]:
df["Textblob"]=list(map(get_textblob_score,list_of_converted_lyr))
df["Vader"]=list(map(get_vader_score,list_of_converted_lyr))
df["Watson"]=list(map(get_watson_score,list_of_converted_lyr))
df

,Textblob,Textblob per line,Textblob nonzero,Vader,Vader per line,Vader nonzero,Watson,Watson per line,Watson nonzero
Lyrics,0.149415,NaN,NaN,0.9599,NaN,NaN,-0.778259,NaN,NaN
SW,0.196875,NaN,NaN,0.9894,NaN,NaN,-0.386954,NaN,NaN
CO,0.149415,NaN,NaN,0.9953,NaN,NaN,-0.773304,NaN,NaN
LM,0.147222,NaN,NaN,0.9659,NaN,NaN,-0.649890,NaN,NaN
SW-CO,0.196875,NaN,NaN,0.9953,NaN,NaN,-0.492278,NaN,NaN
SW-LM,0.186842,NaN,NaN,0.9905,NaN,NaN,-0.289106,NaN,NaN
CO-LM,0.147222,NaN,NaN,0.9960,NaN,NaN,-0.622290,NaN,NaN
SW-CO-LM,0.186842,NaN,NaN,0.9957,NaN,NaN,-0.356586,NaN,NaN


In [130]:
textblob_per_line=[]
for trans in list_of_converted_lyr_arr:
  textblob_per_line.append(list(map(get_textblob_score,trans)))

df["Textblob per line"]=list(map(np.mean,textblob_per_line))
#textblob_per_line

In [131]:
vader_per_line=[]
for trans in list_of_converted_lyr_arr:
  vader_per_line.append(list(map(get_vader_score,trans)))

df["Vader per line"]=list(map(np.mean,vader_per_line))
#vader_per_line

In [132]:
def watson_min_words_arr(arr): #watson has minimum requirement to perform sentiment analysis
  x="" 
  watson_arr=[]
  for line in arr:
    if len(line.split())<4:
      x=x+line
      continue
    if x!= "" : line=x+" "+line
    x=""
    watson_arr.append(line)
  return watson_arr 

converted_watson_arr=list(map(watson_min_words_arr,list_of_converted_lyr_arr))

In [133]:
watson_per_line=[]
for trans in converted_watson_arr:
  watson_per_line.append(list(map(get_watson_score,trans)))

df["Watson per line"]=list(map(np.mean,watson_per_line))
#watson_per_line

In [134]:
def mean_without_zero(arr):
  arr = [x for x in arr if x != 0]
  return np.mean(arr)

In [135]:
df["Textblob nonzero"]=list(map(mean_without_zero,textblob_per_line))
df["Vader nonzero"]=list(map(mean_without_zero,vader_per_line))
df["Watson nonzero"]=list(map(mean_without_zero,watson_per_line))

In [136]:
df

,Textblob,Textblob per line,Textblob nonzero,Vader,Vader per line,Vader nonzero,Watson,Watson per line,Watson nonzero
Lyrics,0.149415,0.046778,0.233889,0.9599,0.095550,0.281029,-0.778259,-0.118538,-0.269406
SW,0.196875,0.053000,0.331250,0.9894,0.100832,0.315100,-0.386954,-0.020349,-0.063953
CO,0.149415,0.046778,0.233889,0.9953,0.098372,0.289329,-0.773304,-0.111188,-0.252699
LM,0.147222,0.054778,0.248990,0.9659,0.099942,0.293947,-0.649890,-0.030253,-0.072030
SW-CO,0.196875,0.053000,0.331250,0.9953,0.103654,0.323919,-0.492278,-0.018453,-0.059314
SW-LM,0.186842,0.061000,0.338889,0.9905,0.107898,0.337181,-0.289106,0.010841,0.029814
CO-LM,0.147222,0.054778,0.248990,0.9960,0.102764,0.302247,-0.622290,-0.025782,-0.061385
SW-CO-LM,0.186842,0.061000,0.338889,0.9957,0.110720,0.346000,-0.356586,0.010238,0.028794


Kinda done with this part. Looks okay for now. Now we got to see if we can get an mp3 version of the song and see if the song sounds sad or happy. 